## DO NOT USE FOR LOOP ON number of samples N but ONLY ON number of classes C

In [69]:
import numpy as np
from sklearn.datasets import load_iris, load_digits, load_digits
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.preprocessing import Binarizer

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Gaussian Discriminant Analysis

In [70]:
data = load_iris()
X_train, y_train = data.data, data.target

In [71]:
def compute_priors(X, y):
    """
    Prior probability for each class 
    
    Inputs:
    - X: array of shape (N, D) 
    - y: array of shape (N,) 

    Returns:
    - priors : array of shape (C,)
    """
    C = (np.max(y) + 1)
    priors = np.zeros(C)
    # YOUR CODE HERE
    for i in range(C):
        priors[i] = sum(y[y==i].shape)/y.shape[0]
    
    return priors

In [72]:
sk_model = QuadraticDiscriminantAnalysis()
sk_model.fit(X_train, y_train)

priors = compute_priors(X_train, y_train)
error = rel_error(sk_model.priors_, priors)
print(error)
assert  error < 1e-12

0.0


In [76]:
def compute_means(X, y):
    """
    Mean estimate for each class, NO FOR LOOP ON number of samples N but ONLY ON number of classes C
    
    Inputs:
    - X: array of shape (N, D) xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    - y: array of shape (N,) 

    Returns:
    - means : array of shape (C, D)
    """
    N, D = X.shape    
    C = (np.max(y) + 1)
    means = np.zeros((C, D))
    # YOUR CODE HERE
    for i in range(C):
        indices_of_rows = np.where(np.isin(y,i))
        X_subset = X[indices_of_rows]
        
        means[i] = (np.sum(X_subset,axis=0) / float(len(X_subset)))
        
    return means
    


In [77]:
sk_model = QuadraticDiscriminantAnalysis()
sk_model.fit(X_train, y_train)

means = compute_means(X_train, y_train)
error = rel_error(sk_model.means_, means)
print(error)
assert  error < 1e-12

0.0


In [78]:
#def cov(rows,mean):
 #   return (rows.reshape(-1,1)-mean).dot((rows.reshape(-1,1)-mean).T)

In [79]:
def compute_sigmas_gda(X, y, means):
    """
    Covariance estimate for each class, NO FOR LOOP ON number of samples N but ONLY ON number of classes C
    DO NOT USE np.cov
    Inputs:
    - X: array of shape (N, D) 
    - y: array of shape (N,) 
    - means: array of shape (C, D)

    Returns:
    - covariances : array of shape (C, D, D)
    """
    N, D = X.shape    
    C = (np.max(y) + 1)
    covariances = np.zeros((C, D, D))
    # YOUR CODE HERE
    for i in range(C):
        indices_of_rows = np.where(np.isin(y,i))
        X_subset = X[indices_of_rows]
        covariances[i] = (1./(X_subset.shape[0]-1)) * ((X_subset - means[i]).T@(X_subset - means[i]))
    
    return covariances;


In [80]:
sk_model = QuadraticDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)

covariances = compute_sigmas_gda(X_train, y_train, sk_model.means_)
error = rel_error(np.asarray(sk_model.covariance_), covariances)
print(error)
assert  error < 1e-12

5.920351755031412e-16


In [81]:
def compute_sigma_lda(X, y, means):
    """
    Covariance estimate for LDA, NO FOR LOOP ON number of samples N but ONLY ON number of classes C
    DO NOT USE np.cov
    Inputs:
    - X: array of shape (N, D) 
    - y: array of shape (N,) 
    - means: array of shape (C, D)

    Returns:
    - covariance : array of shape (D, D)
    """
    N, D = X.shape    
    C = (np.max(y) + 1)
    covariance = np.zeros((D, D))
    # YOUR CODE HERE
    means = compute_means(X_train,y_train)

    for i in range(C):
        X_subset = X[y==i]
        covariance += (X_subset.shape[0]-1)*covariances[i]
    
    return covariance/N;


In [82]:
sk_model = LinearDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)

covariances = compute_sigma_lda(X_train, y_train, sk_model.means_)
error = rel_error(np.asarray(sk_model.covariance_), covariances)
print(error)
assert  error < 1e-12

8.453612124347883e-17


In [94]:
def compute_log_posterior_lda(X, C, priors, means, covariance):
    """
    Covariance log posterior for each class and observation, 
    NO FOR LOOP ON number of samples N but ONLY ON number of classes C
    DO NOT USE scipy or np multivariate gaussian
    Inputs:
    - X: array of shape (N, D) 
    - y: array of shape (N,) 
    - C: number of classes
    - priors : array of shape (C,)
    - means : array of shape (C, D)
    - covariance : array of shape (D, D)

    Returns:
    - log_posterior : array of shape (N, C)
    """
    N, D = X.shape    
    log_posterior = np.zeros((N, C))
    W = np.zeros((C,D))
    b = np.zeros(C)
    # YOUR CODE HERE
    for i in range(C):
        log_posterior[:,i]= np.diag(np.log(priors[i]) - (1/2)*np.log(np.linalg.det(covariance)) - (1/2)*( (X-means[i])@np.linalg.inv(covariance)@ (X-means[i]).T )) 
        
    return log_posterior


In [96]:
# NO TEST FOR LOG-POSTERIOR LDA. Mitambatra eo ambany ny test

In [101]:
def compute_log_posterior_gda(X, C, priors, means, covariances):
    """
    Covariance log posterior for each class and observation, 
    NO FOR LOOP ON number of samples N but ONLY ON number of classes C
    DO NOT USE scipy or np multivariate gaussian
    Inputs:
    - X: array of shape (N, D) 
    - y: array of shape (N,) 
    - C: number of classes
    - priors : array of shape (C,)
    - means : array of shape (C, D)
    - covariances : array of shape (C, D, D)

    Returns:
    - log_posterior : array of shape (N, C)
    """
    N, D = X.shape    
    log_posterior = np.zeros((N, C))
    # YOUR CODE HERE
    for i in range(C):
        log_posterior[:,i]= np.diag(np.log(priors[i]) - (1/2)*np.log(np.linalg.det(covariances[i])) - (1/2)*( (X-means[i])@np.linalg.inv(covariances[i])@ (X-means[i]).T ) )
        
    return log_posterior


In [102]:
sk_model = QuadraticDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)

C = (np.max(y_train) + 1)
log_posterior = compute_log_posterior_gda(X_train, C, sk_model.priors_, sk_model.means_, sk_model.covariance_)
error = rel_error(np.asarray(sk_model._decision_function(X_train)), log_posterior)
print(error)
assert  error < 1e-12

6.605541792973586e-14


In [103]:
class ProbClassifier():
    def fit(self, X, y):
        pass
    
    def compute_log_posterior(self, X):
        pass
    
    def predict(self, X):
        log_post = self.compute_log_posterior(X)
        # YOUR CODE HERE
        return np.argmax(log_post,axis=1)
    
    def predict_proba(self, X):
        log_post = self.compute_log_posterior(X)
        # YOUR CODE HERE
        N = np.exp(log_post).sum(axis=1).shape[0]
        
        return np.exp(log_post)/np.exp(log_post).sum(axis=1).reshape((N,1))


In [104]:
class LDA(ProbClassifier):
    def __init__(self):
        self.priors = None
        self.means = None
        self.cov = None
        self.C = None
    
    def fit(self, X, y):
        self.C = (np.max(y) + 1)
        # YOUR CODE HERE
        self.priors = compute_priors(X, y)
        self.means = compute_means(X, y)
        self.cov = compute_sigma_lda(X, y, self.means)
        #raise NotImplementedError()
    
    def compute_log_posterior(self, X):
        # YOUR CODE HERE
        return compute_log_posterior_lda(X, self.C, self.priors, self.means, self.cov)


In [105]:
sk_model = LinearDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

lda = LDA()
lda.fit(X_train, y_train)
pred = lda.predict(X_train)

assert (sk_pred == pred).all()
print("Accuracy scikit-learn : ", accuracy_score(y_train, sk_pred))
print("Your Accuracy : ", accuracy_score(y_train, pred))

<ipython-input-95-eb4606b1b2e9>:26: RuntimeWarning: divide by zero encountered in log
  b= (1/2)*np.log(np.linalg.det(covariance))


LinAlgError: Singular matrix

In [106]:
class QDA(ProbClassifier):
    def __init__(self):
        self.priors = None
        self.means = None
        self.cov = None
        self.C = None
    
    def fit(self, X, y):
        self.C = (np.max(y) + 1)
        # YOUR CODE HERE
        self.priors = compute_priors(X, y)
        self.means = compute_means(X, y)
        self.cov = compute_sigmas_gda(X, y, self.means)
    
    def compute_log_posterior(self, X):
        # YOUR CODE HERE
        return compute_log_posterior_gda(X, self.C, self.priors, self.means, self.cov)


In [107]:
sk_model = QuadraticDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

qda = QDA()
qda.fit(X_train, y_train)
pred = qda.predict(X_train)

assert (sk_pred == pred).all()
print("Accuracy scikit-learn : ", accuracy_score(y_train, sk_pred))
print("Your Accuracy : ", accuracy_score(y_train, pred))

Accuracy scikit-learn :  0.98
Your Accuracy :  0.98


In [108]:
sk_model = QuadraticDiscriminantAnalysis(store_covariance=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict_proba(X_train)

qda = QDA()
qda.fit(X_train, y_train)
pred = qda.predict_proba(X_train)

error = rel_error(pred, sk_pred)
print(error)
assert error < 1e-12

9.111122555721264e-15


# Naive Bayes Classifiers

##  Bernouilli Naive Bayes

In [109]:
data = load_digits()
X_train2, y_train2 = data.data, data.target
X_train2_transf = Binarizer().fit_transform(X_train2)

In [110]:
class BernouilliNaiveBayes(ProbClassifier):
    def __init__(self):
        self.priors = None
        self.C = None
        self.theta = None
    
    def fit(self, X, y):
        """
        Estimate the parameter theta
        NO FOR LOOP ON number of samples N but ONLY ON number of classes C
        DO NOT USE scipy or np density
        """
        N, D = X.shape
        self.C = (np.max(y) + 1)
        self.theta = np.zeros((D, self.C,))
        # YOUR CODE HERE
        self.priors = np.zeros((self.C))
        for i in range(self.C):
            X_subset = X[y==i]
            Ni = X_subset.shape[0]
            self.priors[i] = Ni/N
            self.theta[:,i] = np.sum(X_subset,axis=0)/Ni
    
    def compute_log_posterior(self, X):
        N, D = X.shape
        log_post = np.zeros((N,self.C))
        # YOUR CODE HERE
        eps = 1e-11
        log_post = (X @ np.log(self.theta + eps)) + ((1-X) @ np.log(1-self.theta + eps)) + np.log(self.priors)
        
        return log_post


In [111]:
sk_model = BernoulliNB()
sk_model.fit(X_train2_transf, y_train2)
sk_pred = sk_model.predict(X_train2_transf)

model = BernouilliNaiveBayes()
model.fit(X_train2_transf, y_train2)
pred = model.predict(X_train2_transf)

sk_acc = accuracy_score(y_train2, sk_pred)
model_acc = accuracy_score(y_train2, pred)
print("Accuracy scikit-learn : ", sk_acc)
print("Your Accuracy : ", model_acc)
assert sk_acc - model_acc < 0.01

Accuracy scikit-learn :  0.8636616583194212
Your Accuracy :  0.8742348358375069


## Gaussian Naive Bayes

In [113]:
class GaussianNaiveBayes(ProbClassifier):
    def __init__(self):
        self.priors = None
        self.C = None
        self.mu = None
        self.sigma = None
    
    def fit(self, X, y):
        """
        Estimate the parameters mu and sigma
        NO FOR LOOP ON number of samples N but ONLY ON number of classes C
        DO NOT USE scipy or np density
        """
        N, D = X.shape
        self.C = (np.max(y) + 1)
        self.sigma = np.zeros((D, self.C))
        self.mu = np.zeros((D,self.C))
        # YOUR CODE HERE
        self.priors = np.zeros(self.C)
        for i in range(self.C):
            Ni = np.sum(y==i)
            self.priors[i] = Ni/N
            self.mu[:,i] = np.sum(X[y==i],axis=0)/Ni
            self.sigma[:,i] =  np.sum((1/Ni)*(X[y==i] - (np.sum(X[y==i], axis=0)/Ni))**2, axis=0)
    
    def compute_log_posterior(self, X):
        N, D = X.shape
        log_post = np.zeros((N,self.C))
        # YOUR CODE HERE
        for j in range(self.C):
            log_post[:, j] = np.log(self.priors[j]) + -(1/2)*np.sum(np.log(2*np.pi* self.sigma[:,j].T)+((X - self.mu[:,j].T)**2)/self.sigma[:,j].T, axis=1)
        
        return log_post


In [114]:
sk_model = GaussianNB()
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = GaussianNaiveBayes()
model.fit(X_train, y_train)
pred = model.predict(X_train)

sk_acc = accuracy_score(y_train, sk_pred)
model_acc = accuracy_score(y_train, pred)
print("Accuracy scikit-learn : ", sk_acc)
print("Your Accuracy : ", model_acc)
assert sk_acc - model_acc < 0.01

Accuracy scikit-learn :  0.96
Your Accuracy :  0.96
